In [1]:
import os
import cv2

In [2]:
# Set the paths to your dataset and output directories
dataset_root = r'C:\Users\Zainab Shafique\Desktop\Human Activity Recognition\Actions'
output_root = r'C:\Users\Zainab Shafique\Desktop\Human Activity Recognition\OUTPUT'


In [3]:
# List of classes
classes = ['backflip', 'clapping','applauding','bandaging','brushing teeth','cheerleading',
           'crawling baby','crying','digging','drawing']

# Create output directories for each class if they don't exist
for class_name in classes:
    class_output_dir = os.path.join(output_root, class_name)
    os.makedirs(class_output_dir, exist_ok=True)


In [4]:
# Function to extract frames from a video and save them as images
def extract_frames(video_path, output_dir, class_name):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_filename = os.path.join(output_dir, f'frame_{i:04d}.jpg')
        cv2.imwrite(frame_filename, frame)

    cap.release()


In [5]:
# Loop through each class
for class_name in classes:
    class_data_dir = os.path.join(dataset_root, class_name)
    class_output_dir = os.path.join(output_root, class_name)

    # List all video files in the class data directory
    video_files = [f for f in os.listdir(class_data_dir) if f.endswith('.mp4')]

    # Loop through each video file in the class
    for video_file in video_files:
        video_path = os.path.join(class_data_dir, video_file)

        # Create a subdirectory for each video in the class output directory
        video_output_dir = os.path.join(class_output_dir, os.path.splitext(video_file)[0])
        os.makedirs(video_output_dir, exist_ok=True)

        # Extract frames from the video and save them in the video's output directory
        extract_frames(video_path, video_output_dir, class_name)

print("Frame extraction complete.")


Frame extraction complete.


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, LSTM, Dense
from tensorflow.keras.layers import TimeDistributed


# Define the model
model = Sequential()

# Define the number of frames, image dimensions, and channels
frames = 16  # Adjust this to the number of frames in your video clips
height = 128
width = 128
channels = 3  # 3 for RGB, 1 for grayscale

# Convolutional layers
model.add(Conv3D(64, kernel_size=(3, 3, 3), input_shape=(frames, height, width, channels), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# LSTM layer
model.add(TimeDistributed(Flatten()))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=True))

# Fully connected layers
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model
model.summary()


NameError: name 'num_classes' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Define the number of classes and the image dimensions
num_classes = 10
frame_height = 64
frame_width = 64
  # Adjust this based on your dataset



# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Define data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Rescale pixel values to the range [0, 1]
    rotation_range=20,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift the width of images
    height_shift_range=0.2,  # Randomly shift the height of images
    horizontal_flip=True,  # Randomly flip images horizontally
    validation_split=0.2  # Split data into training and validation sets
)

batch_size = 32
train_generator = train_datagen.flow_from_directory(
    r'C:\Users\Zainab Shafique\Desktop\Human Activity Recognition\OUTPUT',
    target_size=(frame_width, frame_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    r'C:\Users\Zainab Shafique\Desktop\Human Activity Recognition\OUTPUT',
    target_size=(frame_width, frame_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# # Train the model
# epochs = 20  # Adjust the number of epochs as needed
# history = model.fit(
#     train_generator,
#     epochs=epochs,
#     validation_data=validation_generator
# )

# # Save the trained model
# model.save('my_convlstm_model.h5')


In [ ]:
#prediction code

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
import tensorflow_hub as hub
# Load the pre-trained model
model = load_model('my_fine_tuned_resnet_model.h5')

# Define a function to preprocess an image for prediction
def preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize pixel values to [0, 1]
    return img

# Define a function to make predictions
def predict_action(image_path):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    predictions = model.predict(img)
    class_indices = {0: "back flipping", 1: "clapping"}
    predicted_class = class_indices[np.argmax(predictions)]
    confidence = predictions[0][np.argmax(predictions)]
    return predicted_class, confidence

# Open the video file
cap = cv2.VideoCapture(r"C:\Users\Zainab Shafique\Desktop\Human Activity Recognitionvideo.mp4")

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Preprocess the frame (resize, normalize, stack)
    frame = cv2.resize(frame, (224, 224))  # Adjust the size as per your model
    frame = frame / 255.0  # Normalize pixel values
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    
    # Make a prediction
    predictions = model.predict(frame)
    
    # Get the predicted action class
    predicted_class = action_classes[np.argmax(predictions)]
    
    # Display the predicted class on the frame
    cv2.putText(frame, predicted_class, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Display the frame
    cv2.imshow("Action Recognition", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



# # Example usage for a single image
# image_path = r'C:\Users\Zainab Shafique\Desktop\Human Activity Recognition\OUTPUT\clapping\Cf2Ta-e0ghI/frame_0064.jpg'
# predicted_class, confidence = predict_action(image_path)
# print(f"Predicted Class: {predicted_class}")
# print(f"Confidence: {confidence:.2f}")
